In [37]:
import numpy as np
from evol import Population, Evolution
import attr
import typing
from itertools import combinations

In [35]:
@attr.s(kw_only=True)
class Motor: 
    type_: str = attr.ib()
    voltage: int = attr.ib()
    
    @type_.validator
    def valid_motor_type(self, attribute, value):
        valid_motor_types = ["step_motor", "servo_motor"]
        if value not in valid_motor_types:
            raise ValueError(f"`{value}` is an invalid motor type. Valid motor types are {valid_motor_types}")
    @voltage.validator
    def valid_voltage(self, attribute, value):
        valid_voltage_values = [5, 9, 12, 24]
        if value not in valid_voltage_values:
            raise ValueError(f"`{value}` is an invalid voltage value. Valid voltage values are {valid_voltage_values}")

In [43]:
@attr.s(kw_only=True)
class PowerSource:
    type_: str = attr.ib()
    voltage: int = attr.ib()
    @type_.validator
    def valid_power_source_type(self, attribute, value):
        valid_power_source_types = ["nickel_cadmium", "lithium_ion", "solar_panel", "fusion_reactor"]
        if value not in valid_power_source_types:
            raise ValueError(f"`{value}` is an invalid power source type. Valid power source types are {valid_power_source_types}")
    @voltage.validator
    def valid_voltage(self, attribute, value):
        valid_voltage_values = [12, 24]
        if value not in valid_voltage_values:
            raise ValueError(f"`{value}` is an invalid voltage value. Valid voltage values are {valid_voltage_values}")    
    

In [44]:
PowerSource(type_="lithium_ion", voltage=12)

PowerSource(type_='lithium_ion', voltage=12)

In [7]:
def fitness(individual):
    return individual.range + individual.power - individual.weight